In [4]:
from gurobipy import *
import gurobipy as gp

# type declaration
from typing import Dict, List

# %load_ext nb_black
%load_ext lab_black

In [22]:
oil_fields: List[str] = ["Texas", "California"]
refineries: List[str] = ["NewOrleans", "Charlton", "Seattle"]
distribution_centers: List[str] = ["Pittsburgh", "Atlanta"]

combinations_oil_refineries: List[str] = []
for oil in oil_fields:
    for refinery in refineries:
        combinations_oil_refineries.append(oil + "_" + refinery)

combinations_oil_refineries_distributioncenters: List[str] = []
for combi in combinations_oil_refineries:
    for distribution_center in distribution_centers:
        combinations_oil_refineries_distributioncenters.append(
            combi + "_" + distribution_center
        )

In [23]:
combinations_oil_refineries: Dict[str, int] = {}
prices_o_r: List[int] = [11, 7, 2, 7, 4, 8]
i = 0
for oil in oil_fields:
    for refinery in refineries:
        combinations_oil_refineries[oil + "_" + refinery] = prices_o_r[i]
        i += 1

In [ ]:
combinations_oil_refineries_distributioncenters: List[str] = {}
prices_o_r: List[int] = [11, 7, 2, 7, 4, 8]


for combi in combinations_oil_refineries.keys():
    for distribution_center in distribution_centers:
        combinations_oil_refineries_distributioncenters.append(
            combi + "_" + distribution_center
        )

In [25]:
combinations_oil_refineries

{'Texas_NewOrleans': 11,
 'Texas_Charlton': 7,
 'Texas_Seattle': 2,
 'California_NewOrleans': 7,
 'California_Charlton': 4,
 'California_Seattle': 8}

In [ ]:
combinations_oil_refineries_distributioncenters: List[str] = {}
prices_o_r: List[int] = [11, 7, 2, 7, 4, 8]


for combi in combinations_oil_refineries.keys():
    for distribution_center in distribution_centers:
        combinations_oil_refineries_distributioncenters.append(
            combi + "_" + distribution_center
        )

In [1]:
combinations_oil_refineries_distributioncenters : Dict[str, int] = {
('Texas_NewOrleans_Pittsburgh'): 11+11,
('Texas_Charlton_Pittsburgh'): 7+7,
('Texas_Seattle_Pittsburgh'): 2+5,
('Texas_NewOrleans_Atlanta'): 11+7,
('Texas_Charlton_Atlanta'): 7+4,
('Texas_Seattle_Atlanta'): 2 +3 ,
('California_NewOrleans_Pittsburgh'): 7+11,
('California_Charlton_Pittsburgh'): 4+7,
('California_Seattle_Pittsburgh'): 8+5,
('California_NewOrleans_Atlanta'): 7+7,
('California_Charlton_Atlanta'): 4+4,
('California_Seattle_Atlanta'): 8+3 }


In [155]:
combinations_oil_refineries_distributioncenters: Dict[tuple, int] = {
    ("T", "N", "P"): 11 + 11,
    ("T", "C", "P"): 7 + 7,
    ("T", "S", "P"): 2 + 5,
    ("T", "N", "A"): 11 + 7,
    ("T", "C", "A"): 7 + 4,
    ("T", "S", "A"): 2 + 3,
    ("CA", "N", "P"): 7 + 11,
    ("CA", "C", "P"): 4 + 7,
    ("CA", "S", "P"): 8 + 5,
    ("CA", "N", "A"): 7 + 7,
    ("CA", "C", "A"): 4 + 4,
    ("CA", "S", "A"): 8 + 3,
}
combinations_oil_refineries_distributioncenters_list = list(
    combinations_oil_refineries_distributioncenters.keys()
)

oil_fields: List[str] = ["T", "CA"]
refineries: List[str] = ["NO", "C", "S"]
distribution_centers: List[str] = ["P", "A"]

In [156]:
# oil_field_limits
oil_field_limits: Dict[str, int] = {"CA": 50000, "T": 10000}

# demand limits
demand_limits: Dict[str, int] = {"P": 20000, "A": 75000}

In [157]:
# set the decision function
model = gp.Model()
model.ModelSense = gp.GRB.MINIMIZE


# set the objective function
obj = {}
for combi in combinations_oil_refineries_distributioncenters_list:
    obj[combi] = model.addVar(
        lb=0, obj=combinations_oil_refineries_distributioncenters[combi]
    )
    print(combi)

('T', 'N', 'P')
('T', 'C', 'P')
('T', 'S', 'P')
('T', 'N', 'A')
('T', 'C', 'A')
('T', 'S', 'A')
('CA', 'N', 'P')
('CA', 'C', 'P')
('CA', 'S', 'P')
('CA', 'N', 'A')
('CA', 'C', 'A')
('CA', 'S', 'A')


In [158]:
# CONSTRAINT FOR SUPPLY: essenetially: for every combintion with T at beginning; this is <= 10000; and <= for CA
for oil_field in oil_field_limits.keys():
    lhs = 0
    for objective_r in obj.keys():
        if objective_r[0] == oil_field:
            lhs += obj[objective_r]
    model.addConstr(lhs <= oil_field_limits[oil_field])

In [159]:
# Constraint for demand
for distro in demand_limits.keys():
    lhs = 0
    for objective_r in obj.keys():
        if objective_r[-1] == distro:
            lhs += obj[objective_r]
    model.addConstr(lhs == demand_limits[distro])

In [160]:
model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value", model.objval)
model.printAttr("X")

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 12 columns and 24 nonzeros
Model fingerprint: 0xcacbdc06
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 8e+04]
Presolve removed 0 rows and 8 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible model
something went wrong


AttributeError: Unable to retrieve attribute 'objval'

In [161]:
model.write("problem.lp")
with open("problem.lp") as f:
    print(f.read())

\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  22 C0 + 14 C1 + 7 C2 + 18 C3 + 11 C4 + 5 C5 + 18 C6 + 11 C7 + 13 C8
   + 14 C9 + 8 C10 + 11 C11
Subject To
 R0: C6 + C7 + C8 + C9 + C10 + C11 <= 50000
 R1: C0 + C1 + C2 + C3 + C4 + C5 <= 10000
 R2: C0 + C1 + C2 + C6 + C7 + C8 = 20000
 R3: C3 + C4 + C5 + C9 + C10 + C11 = 75000
Bounds
End



In [69]:
# the constraint; for T and CA there is only so much supply!

model.addConstrs(
    gp.quicksum(obj[oil_field, refinery, distrib] for refinery in refineries)
    == oil_field_limits[oil_field]
    for oil_field in oil_fields
)

SyntaxError: Generator expression must be parenthesized (1339337365.py, line 4)

In [ ]:
model.addConstrs(gp.quicksum(x[g,p] for p in plants)==supplies[g] 
                 for g in groves)

In [ ]:
groves = ["A", "B", "C"]
plants = ["D", "E", "F"]
distances = {
    ("A", "D"): 21,
    ("A", "E"): 50,
    ("A", "F"): 40,
    ("B", "D"): 35,
    ("B", "E"): 30,
    ("B", "F"): 22,
    ("C", "D"): 55,
    ("C", "E"): 20,
    ("C", "F"): 25,
}
costPerMile = 1.5
capacities = {"D": 2000, "E": 6000, "F": 2250}
supplies = {"A": 2750, "B": 4000, "C": 3000}

In [ ]:
model.addConstrs(gp.quicksum(x[g,p] for p in plants)==supplies[g] 
                 for g in groves)



lhs = 0
for i in Bonds:
    lhs += bonds_obj[i]
model.addConstr(lhs == upper_bound)              

In [59]:
model.write("problem.lp")
with open("problem.lp") as f:
    print(f.read())

\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  22 C0 + 14 C1 + 7 C2 + 18 C3 + 11 C4 + 5 C5 + 13 C6 + 11 C7 + 14 C8 + 8 C9
   + 11 C10
Subject To
Bounds
End



In [122]:
groves = ["A", "B", "C"]
plants = ["D", "E", "F"]
distances = {
    ("A", "D"): 21,
    ("A", "E"): 50,
    ("A", "F"): 40,
    ("B", "D"): 35,
    ("B", "E"): 30,
    ("B", "F"): 22,
    ("C", "D"): 55,
    ("C", "E"): 20,
    ("C", "F"): 25,
}
costPerMile = 1.5
capacities = {"D": 2000, "E": 6000, "F": 2250}
supplies = {"A": 2750, "B": 4000, "C": 3000}

model = gp.Model()
x = model.addVars(distances, name="flow")
model.setObjective(
    costPerMile * gp.quicksum(distances[g, p] * x[g, p] for g in groves for p in plants)
)
model.addConstrs(gp.quicksum(x[g, p] for p in plants) == supplies[g] for g in groves)
model.addConstrs(gp.quicksum(x[g, p] for g in groves) <= capacities[p] for p in plants)

{'D': <gurobi.Constr *Awaiting Model Update*>,
 'E': <gurobi.Constr *Awaiting Model Update*>,
 'F': <gurobi.Constr *Awaiting Model Update*>}

In [123]:
model.write("problem.lp")
with open("problem.lp") as f:
    print(f.read())

\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  31.5 flow[A,D] + 75 flow[A,E] + 60 flow[A,F] + 52.5 flow[B,D]
   + 45 flow[B,E] + 33 flow[B,F] + 82.5 flow[C,D] + 30 flow[C,E]
   + 37.5 flow[C,F]
Subject To
 R0: flow[A,D] + flow[A,E] + flow[A,F] = 2750
 R1: flow[B,D] + flow[B,E] + flow[B,F] = 4000
 R2: flow[C,D] + flow[C,E] + flow[C,F] = 3000
 R3: flow[A,D] + flow[B,D] + flow[C,D] <= 2000
 R4: flow[A,E] + flow[B,E] + flow[C,E] <= 6000
 R5: flow[A,F] + flow[B,F] + flow[C,F] <= 2250
Bounds
End



In [71]:


model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value", model.objval)
model.printAttr("X")

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0x3b94ed6b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 6e+03]
Presolve time: 0.01s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0862500e+05   2.500000e+03   0.000000e+00      0s
       2    3.6000000e+05   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.600000000e+05
optimal value 360000.0

    Variable            X 
-------------------------
   flow[A,D]         2000 
   flow[A,F]          750 
   flow[B,E]         2500 
   flow[B,F]         1500 
   flow[C,E]         3000 


In [72]:
# add the supply constraint constraint

In [73]:
model.write("problem.lp")
with open("problem.lp") as f:
    print(f.read())

\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  31.5 flow[A,D] + 75 flow[A,E] + 60 flow[A,F] + 52.5 flow[B,D]
   + 45 flow[B,E] + 33 flow[B,F] + 82.5 flow[C,D] + 30 flow[C,E]
   + 37.5 flow[C,F]
Subject To
 R0: flow[A,D] + flow[A,E] + flow[A,F] = 2750
 R1: flow[B,D] + flow[B,E] + flow[B,F] = 4000
 R2: flow[C,D] + flow[C,E] + flow[C,F] = 3000
 R3: flow[A,D] + flow[B,D] + flow[C,D] <= 2000
 R4: flow[A,E] + flow[B,E] + flow[C,E] <= 6000
 R5: flow[A,F] + flow[B,F] + flow[C,F] <= 2250
Bounds
End

